In [81]:
import pandas as pd
import regex as re
import os, sys
import xml.etree.ElementTree as et


In [82]:
path = "data-test"


In [83]:
# a function to walk through all files in a folder and its subfolders
def list_files(dir):                                                                                                  
    r = []                                                                                                            
    subdirs = [x[0] for x in os.walk(dir)]                                                                            
    for subdir in subdirs:                                                                                            
        files = os.walk(subdir).__next__()[2]                                                                             
        if (len(files) > 0):                                                                                          
            for file in files:                                                                                        
                r.append(os.path.join(subdir, file))                                                                         
    return r


In [84]:
# processing xml
def read_xml(document, index=0):

  # add filename to dataframe
  m = re.search(r"\\(?!.*\\)(.*)((\.xml))", document)
  filename_match = m.group(0)
  filename = re.sub("[\\\/\'\>]", "", filename_match)
  filename = re.sub("\.\w+", "", filename)
  df.loc[index, 'file'] = filename
  
  print("Preprocessing", filename, "...")

  # creating a pdf file object
  tree = et.parse(document)
  root = tree.getroot()
  
  # Get Id
  df.loc[index, 'id'] = root[0].attrib['Id']

  for company in root.findall('companyName'):
    df.loc[index, 'company'] = company.text

  for ticker in root.findall('companyTicker'):
    df.loc[index, 'ticker'] = ticker.text

  for date in root.findall('startDate'):
    df.loc[index, 'date'] = date.text

  df['date'] = pd.to_datetime(df['date'])
  df['year'] = df['date'].dt.year

  return


In [85]:
# assign relative directory
directory = os.path.join(sys.path[0], path + "\\raw") ### INPUT FOLDER HERE ###

# list files in directory
files_in_dir = list_files(directory)

# count files in directory
print("Your input directory is:", directory, "number of files:",len(files_in_dir))

# initialize dataframe to hold documents
df = pd.DataFrame(columns=['file'])

# iterate over files in the directory
i = 0
for f in files_in_dir:
    if f.lower().endswith('.xml'):
        read_xml(f,i)
    i = i + 1


Your input directory is: c:\Users\parti\GitHub\index-construction\data-test\raw number of files: 20
Preprocessing 562506_T ...
Preprocessing 608796_T ...
Preprocessing 612885_T ...
Preprocessing 615592_T ...
Preprocessing 617166_T ...
Preprocessing 625306_T ...
Preprocessing 625307_T ...
Preprocessing 634760_T ...
Preprocessing 643603_T ...
Preprocessing 646332_T ...


,file,id,company,ticker,date,year
10,562506_T,562506.F,ODP Corp,ODP.OQ,2001-11-28 13:00:00+00:00,2001
11,608796_T,608796.F,Alexander & Baldwin Inc,\n,2002-04-18 19:00:00+00:00,2002
12,612885_T,612885.F,Anaren Inc,ANEN.O^B14,2002-04-25 21:00:00+00:00,2002
13,615592_T,615592.F,ODP Corp,ODP.OQ,2002-04-17 12:00:00+00:00,2002
14,617166_T,617166.F,Glencore Canada Corp,FAL_pa.TO^G08,2002-04-24 20:00:00+00:00,2002
15,625306_T,625306.F,Unilever NV,UNA.AS^K20,2002-04-26 07:45:00+00:00,2002
16,625307_T,625307.F,Unilever NV,UNA.AS^K20,2002-04-26 14:30:00+00:00,2002
17,634760_T,634760.F,Glencore Canada Corp,FAL_pa.TO^G08,2002-05-28 20:00:00+00:00,2002
18,643603_T,643603.F,E*Trade Financial Corp,ETFC.OQ^J20,2002-07-17 21:00:00+00:00,2002
19,646332_T,646332.F,ODP Corp,ODP.OQ,2002-07-17 12:00:00+00:00,2002


In [86]:
# Metadata from dataframe as csv
df = df[['id', 'company', 'year']]
df.columns = ['document_id', 'firm_id', 'time']
df.reset_index(drop=True, inplace=True)
df.to_csv(path+'input\\id2firms.csv')


In [ ]:
print("\nDone!")


In [ ]:
# # Testing

# # Passing the path of the xml document to enable the parsing process
# tree = et.parse('data-test\\raw\\xml\\562506_T.xml')
# # print(tree)

# # getting the parent tag of the xml document
# root = tree.getroot()

# # loop
# for child in root:
#     print(child.tag, child.attrib, child.text)

# # Get Id
# print(root[0].attrib['Id'])

# # # printing the root (parent) tag of the xml document, along with its memory location
# # print(root)
# # print(root.attrib)

# # # printing the attributes of the first tag from the parent
# # print(root[0].attrib)
# # print(root[0][1].attrib)
# # print(root[0][1].text)

# # # loop
# # for child in root:
# #     print(child.tag, child.attrib, child.text)

# for company in root.findall('companyName'):
#     print(company.text)

# for ticker in root.findall('companyTicker'):
#     print(ticker.text)

# for date in root.findall('startDate'):
#     print(date.text)
